In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df=pd.read_csv('car data.csv')
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [10]:
#check for null values
df.isnull().any()


Car_Name         False
Year             False
Selling_Price    False
Present_Price    False
Kms_Driven       False
Fuel_Type        False
Seller_Type      False
Transmission     False
Owner            False
dtype: bool

In [12]:
df.dtypes

Car_Name          object
Year               int64
Selling_Price    float64
Present_Price    float64
Kms_Driven         int64
Fuel_Type         object
Seller_Type       object
Transmission      object
Owner              int64
dtype: object

In [15]:
#split the independent and dependent variables
X=df.drop(['Selling_Price','Car_Name'],1)
y=df['Selling_Price']

In [17]:
X
y.head()


0    3.35
1    4.75
2    7.25
3    2.85
4    4.60
Name: Selling_Price, dtype: float64

In [19]:
X.dtypes

Year               int64
Present_Price    float64
Kms_Driven         int64
Fuel_Type         object
Seller_Type       object
Transmission      object
Owner              int64
dtype: object

In [21]:
# Using OneHotEncoding to convert categorical variables to Numerical
X=pd.get_dummies(X)
X.head()

,Year,Present_Price,Kms_Driven,Owner,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Seller_Type_Dealer,Seller_Type_Individual,Transmission_Automatic,Transmission_Manual
0,2014,5.59,27000,0,0,0,1,1,0,0,1
1,2013,9.54,43000,0,0,1,0,1,0,0,1
2,2017,9.85,6900,0,0,0,1,1,0,0,1
3,2011,4.15,5200,0,0,0,1,1,0,0,1
4,2014,6.87,42450,0,0,1,0,1,0,0,1


In [22]:
# Normalization of data
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=list(X)
X=scaler.fit_transform(X)

In [23]:
X=pd.DataFrame(X)
X.columns=cols
X.head()

,Year,Present_Price,Kms_Driven,Owner,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Seller_Type_Dealer,Seller_Type_Individual,Transmission_Automatic,Transmission_Manual
0,0.733333,0.057109,0.053053,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,0.666667,0.099913,0.085085,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.933333,0.103273,0.012813,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
3,0.533333,0.041504,0.009409,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,0.733333,0.070980,0.083984,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [24]:
#spliting the model into traing and testing set
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X,y, test_size = 0.25, random_state = 42)

In [35]:
# KNN Model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.metrics import r2_score


In [29]:
knn = KNeighborsRegressor()
params = {"n_neighbors":np.arange(1,14)}
knn_gridsch = GridSearchCV(knn,param_grid=params, cv = 10,scoring="r2")

In [30]:
knn_gridsch.fit(X,y)

GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [31]:
knn_gridsch.best_estimator_

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=3, p=2,
          weights='uniform')

In [39]:
knn = KNeighborsRegressor(n_neighbors=3) # via GridSearchCV we found out best value for n=3

In [40]:
knn.fit(X_train,y_train)
y_pred= knn.predict(X_test)
r2_score(y_test,y_pred)

0.9477150463606092

In [80]:
cv=cross_val_score(knn,X_train,y_train,cv = 10, scoring="r2")
print(np.mean(cv))

0.8073599757949911


---------- Linear Regression-------------------

In [51]:
#Linear Regression Model
from sklearn.linear_model import LinearRegression


In [53]:
lr=LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
r2_score(y_test,y_pred)

0.8770281948906486

In [81]:
cv=cross_val_score(lr,X_train,y_train,cv = 10, scoring="r2")
print(np.mean(cv))

0.8005645326765289


In [72]:
cross_val_score(lr,X_train,y_train,cv = 10, scoring="r2") 

array([0.78605039, 0.80930239, 0.719002  , 0.61550944, 0.9088105 ,
       0.92629828, 0.76298657, 0.7222242 , 0.89060638, 0.86485519])

---------------- Lasso ---------------

In [61]:
#Lasso 
from sklearn.linear_model import Lasso
alphas = np.logspace(-3,3,num=14)

In [64]:
grid = GridSearchCV(estimator=Lasso(), param_grid=dict(alpha=alphas))
grid.fit(X_train,y_train)

print(grid.best_score_)
print(grid.best_estimator_.alpha)

0.8570694290743911
0.024244620170823284


In [67]:
ls=Lasso(alpha = grid.best_estimator_.alpha, normalize = True)
ls.fit(X_train,y_train)
y_pred=ls.predict(X_test)
r2_score(y_test,y_pred)

0.8461981550614369

In [82]:
cv=cross_val_score(ls,X_train,y_train,cv = 10, scoring="r2") 
print(np.mean(cv))

0.8170789054433627


--------------- Ridge---------------------------

In [119]:
#Ridge 
from sklearn.linear_model import Ridge
alphas = np.logspace(-3,3,num=14) 

In [120]:
grid2 = GridSearchCV(estimator=Ridge(), param_grid=dict(alpha=alphas)) 
grid2.fit(X_train,y_train)

print(grid2.best_score_)
print(grid2.best_estimator_.alpha)

0.850803859340955
0.2030917620904737


In [122]:
ridge= Ridge(alpha=grid2.best_estimator_.alpha , normalize = True)
ridge.fit(X_train,y_train)
y_pred=ridge.predict(X_test)
r2_score(y_test,y_pred)



0.8462077706033362

In [123]:
cv=cross_val_score(ridge,X_train,y_train,cv = 10, scoring="r2") 
print(np.mean(cv))

0.8036313388286924


------------------- Decision Tree ------------------------------

In [138]:
# Decision Tree
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
dtc=DecisionTreeClassifier()
dtr=DecisionTreeRegressor()

In [141]:
params = {"criterion":["mse", "mae"],
        "max_depth":np.arange(1,20), 
         "min_samples_split":np.arange(0.01,0.13, 0.01)}

In [143]:
dtr_cv= GridSearchCV(dtr, param_grid=params, cv= 10)
dtr_cv.fit(X,y)

print(dtr_cv.best_params_)
print(dtr_cv.best_score_)



{'criterion': 'mse', 'max_depth': 4, 'min_samples_split': 0.02}
0.04858992601550899


In [145]:
dt= DecisionTreeRegressor(criterion='mse',max_depth=4,min_samples_split=0.02)
dt.fit(X_train,y_train)
y_pred=dt.predict(X_test)
r2_score(y_test,y_pred)

0.9320600460265194

------------------------------------------------------------------------------------------------------------------------

In [84]:
# Feature selection
from sklearn.feature_selection import RFE

In [124]:
rfe = RFE(lr, 5)
fit = rfe.fit(X, y)
print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % (fit.support_))
print("Feature Ranking: %s" % (fit.ranking_))

Num Features: 5
Selected Features: [False  True False False  True  True False  True False  True False]
Feature Ranking: [5 1 6 7 1 1 4 1 3 1 2]


In [110]:
X.columns

Index(['Year', 'Present_Price', 'Kms_Driven', 'Owner', 'Fuel_Type_CNG',
       'Fuel_Type_Diesel', 'Fuel_Type_Petrol', 'Seller_Type_Dealer',
       'Seller_Type_Individual', 'Transmission_Automatic',
       'Transmission_Manual'],
      dtype='object')

In [111]:
features=['Present_Price','Fuel_Type_CNG','Fuel_Type_Diesel','Seller_Type_Dealer','Transmission_Automatic']


In [112]:
X1=X[features]
X1.head()

,Present_Price,Fuel_Type_CNG,Fuel_Type_Diesel,Seller_Type_Dealer,Transmission_Automatic,Transmission_Manual
0,0.057109,0.0,0.0,1.0,0.0,1.0
1,0.099913,0.0,1.0,1.0,0.0,1.0
2,0.103273,0.0,0.0,1.0,0.0,1.0
3,0.041504,0.0,0.0,1.0,0.0,1.0
4,0.070980,0.0,1.0,1.0,0.0,1.0


In [92]:
from sklearn.linear_model import RidgeCV, LassoCV

In [113]:
reg = RidgeCV()
reg.fit(X1, y)
print("Best alpha using built-in RidgeCV: %f" % reg.alpha_)
print(reg.score(X1,y))

Best alpha using built-in RidgeCV: 0.100000
0.8131897544111706


In [118]:
lasso_cv = LassoCV()
lasso_cv.fit(X1, y)
print("Best alpha using built-in LassoCV: %f" % lasso_cv.alpha_)
print(lasso_cv.score(X1,y))

Best alpha using built-in LassoCV: 0.012448
0.8132366862525977
